In [1]:
import pyccl as ccl
print(f"I am using pyccl version {ccl.__version__}")
import numpy as np
from scripts.data_vectors import DataVectors
from scripts.presets import Presets
import scripts.plotting as p
import matplotlib.pyplot as plt
from scripts.data_vector_metrics import DataVectorMetrics

I am using pyccl version 2.8.0


In [2]:
redshift_max = 4
redshift_resolution_1 = 300  # this is the first resolution of redshifts
redshift_resolution_2 = 5000  # this is the second resolution of redshifts
forecast_year = "10"
# Please note that the redshift resolution is the number of redshifts between 0 and redshift_max.
# Additionally, a lot of parametric sweeping is already coded into helper functions 
# so we will not need to worry about that for now.

# We will also investigate ell binning but for this particular exercise,
# we will keep the ell binning at default values.
#ell_min = 20
#ell_max = 2000
#n_ell = 50

# We now initialize the class by only passing the redshift information.
presets1 = Presets(redshift_max=redshift_max,
                  redshift_resolution=redshift_resolution_1,
                  forecast_year=forecast_year)
presets2 = Presets(redshift_max=redshift_max,
                    redshift_resolution=redshift_resolution_2,
                    forecast_year=forecast_year)

redshift1 = presets1.redshift_range
redshift2 = presets2.redshift_range

dv1 = DataVectors(presets1)
dv2 = DataVectors(presets2)
ells1 = dv1.ells
ells2 = dv2.ells

shear_cls_1 = dv1.cosmic_shear_cls(include_all_correlations=True)
shear_cls_2 = dv2.cosmic_shear_cls(include_all_correlations=True)

clu_cls_1 = dv1.galaxy_clustering_cls(include_all_correlations=True)
clu_cls_2 = dv2.galaxy_clustering_cls(include_all_correlations=True)

ggl_cls_1 = dv1.galaxy_galaxy_lensing_cls(include_all_correlations=True)
ggl_cls_2 = dv2.galaxy_galaxy_lensing_cls(include_all_correlations=True)

wl_kernel1 = dv1.get_wl_kernel()
wl_kernel2 = dv2.get_wl_kernel()

nc_kernel1 = dv1.get_nc_kernel()
nc_kernel2 = dv2.get_nc_kernel()

Data saved to data_output/angular_power_spectra/cosmic_shear_correlations_all_y10.npy
Data saved to data_output/angular_power_spectra/cosmic_shear_cls_all_y10_zmax4_zres300_ccl_v2.8.0.npy
Data saved to data_output/angular_power_spectra/cosmic_shear_correlations_all_y10.npy
Data saved to data_output/angular_power_spectra/cosmic_shear_cls_all_y10_zmax4_zres5000_ccl_v2.8.0.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_correlations_all_y10.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_cls_all_y10_zmax4_zres300_ccl_v2.8.0.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_correlations_all_y10.npy
Data saved to data_output/angular_power_spectra/galaxy_clustering_cls_all_y10_zmax4_zres5000_ccl_v2.8.0.npy
Data saved to data_output/angular_power_spectra/galaxy_galaxy_lensing_correlations_all_y10.npy
Data saved to data_output/angular_power_spectra/galaxy_galaxy_lensing_cls_all_y10_zmax4_zres300_ccl_v2.8.0.npy
Data saved to data_o

In [3]:
# Initialize DataVectorMetrics for both redshift resolutions
dvm1 = DataVectorMetrics(presets1)
dvm2 = DataVectorMetrics(presets2)

# Compute chi-squared differences using both methods
delta_chi2_method_1 = dvm1.get_delta_chi2(cls_gc=clu_cls_1, cls_ggl=ggl_cls_1, cls_cs=shear_cls_1,
                                          cls_gc_ref=clu_cls_2, cls_ggl_ref=ggl_cls_2, cls_cs_ref=shear_cls_2)

delta_chi2_method_2 = dvm1.get_delta_chi2robert(cl_gc=clu_cls_1, cl_ggl=ggl_cls_1, cl_cs=shear_cls_1,
                                                cl_gc_1=clu_cls_2, cl_ggl_1=ggl_cls_2, cl_cs_1=shear_cls_2)

# Check if results are consistent
print("Delta Chi-squared (New Method):", delta_chi2_method_1)
print("Delta Chi-squared (Robert's Method):", delta_chi2_method_2)

# Compare the results
difference = np.abs(delta_chi2_method_1 - delta_chi2_method_2)
print("Difference between methods:", difference)


Delta Chi-squared (New Method): 0.0006009402177101176
Delta Chi-squared (Robert's Method): 0.0006009402177101176
Difference between methods: 0.0


In [4]:
%%time
kernel_sweep = dvm1.kernel_peaks_z_resolution_sweep()

CPU times: user 5min 23s, sys: 26.2 s, total: 5min 49s
Wall time: 53.6 s


In [8]:
kernel_sweep[300]["wl"]

[(0.1605351170568562, 7.595413790224368e-06),
 (0.294314381270903, 1.6556020234604254e-05),
 (0.4414715719063545, 2.5143600566470706e-05),
 (0.6153846153846154, 3.6351322276534464e-05),
 (0.9632107023411371, 5.809876776090023e-05)]

In [ ]:
p.plot_kernel_peaks_subplots(kernel_sweep,
                               forecast_year=forecast_year,
                               kernel_type="wl",
                               percentage=0.1,
                               marker_size=5,
                               stability_steps=10,
                               fig_format=".pdf",
                               title_pad=0.95)


In [ ]:
peaks1 = []
peaks2 = []

for i, kernel in enumerate(wl_kernel1):
    peak_index = np.argmax(kernel)      # Index of the maximum value in the kernel
    peak_redshift = redshift1[peak_index]  # Redshift at which the peak occurs
    peak_value = kernel[peak_index]     # Peak value itself
    peaks1.append((peak_redshift, peak_value))

    # Plot each kernel with a marker at the peak
    plt.plot(redshift1, kernel, label=f"Kernel {i}")
    plt.plot(peak_redshift, peak_value, 'o', label=f"Peak {i} (z={peak_redshift:.2f})")
    

plt.xlabel("Redshift")
plt.ylabel("Kernel Value")
#plt.legend()
plt.show()
peaks1


In [ ]:
peaks2 = []

for i, kernel in enumerate(wl_kernel2):
    peak_index = np.argmax(kernel)      # Index of the maximum value in the kernel
    peak_redshift = redshift2[peak_index]  # Redshift at which the peak occurs
    peak_value = kernel[peak_index]     # Peak value itself
    peaks2.append((peak_redshift, peak_value))

    # Plot each kernel with a marker at the peak
    plt.plot(redshift2, kernel, label=f"Kernel {i}")
    plt.plot(peak_redshift, peak_value, 'o', label=f"Peak {i} (z={peak_redshift:.2f})")
    

plt.xlabel("Redshift")
plt.ylabel("Kernel Value")
#plt.legend()
plt.show()


In [ ]:
for i in range(len(peaks1)):
    plt.scatter(peaks1[i][0], peaks1[i][1],)
    plt.scatter(peaks2[i][0], peaks1[i][1], marker="v")

In [ ]:
for i in range(len(peaks1)):
    plt.scatter(peaks1[i][0], peaks1[i][1]/peaks2[i][1],)

In [ ]:
p.compare_two_data_vector_sets_absolute(shear_cls_1, shear_cls_2, ells1, ells2)

In [ ]:
p.compare_two_data_vector_sets_relative(shear_cls_1,
                                        shear_cls_2,
                                        ells1,
                                        label_1=f"$n_z={redshift_resolution_1}$", 
                                        label_2=f"$n_z={redshift_resolution_2}$")

In [ ]:
p.compare_two_data_vector_sets_relative(clu_cls_1,
                                        clu_cls_2,
                                        ells1,
                                        label_1=f"$n_z={redshift_resolution_1}$",
                                        label_2=f"$n_z={redshift_resolution_2}$")